In [6]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import glob
import os

factors = ['diff_orb', 'diff_tov', 'diff_3p%', 'diff_2p%', 'diff_ft%', 'diff_fta', 'diff_3pa']
pattern = '/Users/moneysniper/Documents/NBA_analysis_project/gamelogs/regular_season_*.csv'
csv_files = glob.glob(pattern)
csv_files.append('/Users/moneysniper/Documents/NBA_analysis_project/gamelogs/regular_season.csv')
all_data = pd.DataFrame()

for csv in csv_files:
    df = pd.read_csv(csv)
    df['year'] = int(csv.split('_')[-1].split('.')[0]) if 'season_' in csv else 'all_year'
    all_data = pd.concat([all_data, df])

grouped = all_data.groupby('year')

pseudo_r_squared = {}

for name, group in grouped:
    df = group.dropna(subset = factors + ['won'])
    df = df[df['Unnamed: 0'] % 2 != 0]

    pseudo_r_squared[name] = {}
    
    for excluded_factor in factors:
        factors_1 = factors.copy()
        factors_1.remove(excluded_factor)
        
        X = df[factors_1]
        y = df['won']
        
        X = sm.add_constant(X)
        model = sm.Logit(y, X)
        result_1 = model.fit()

        print(result_1.summary(title = f'{name} excluding {excluded_factor}'))
        
        pseudo_r_squared[name][excluded_factor] = result_1.prsquared

pseudo_r_squared_df = pd.DataFrame(pseudo_r_squared)
pseudo_r_squared_df.to_csv('pseudo_r_sqaured.csv')

Optimization terminated successfully.
         Current function value: 0.259056
         Iterations 8
                           2016 excluding diff_orb                            
Dep. Variable:                    won   No. Observations:                 1230
Model:                          Logit   Df Residuals:                     1223
Method:                           MLE   Df Model:                            6
Date:                Thu, 08 Aug 2024   Pseudo R-squ.:                  0.6175
Time:                        11:37:55   Log-Likelihood:                -318.64
converged:                       True   LL-Null:                       -833.15
Covariance Type:            nonrobust   LLR p-value:                4.720e-219
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2445      0.103      2.378      0.017       0.043       0.446
diff_tov      -0.2674      0.